# **DESCRIPTA.AI** -> Processor Code (processor.py)

A Python service to extract images and LaTeX content from ZIP archives, generate AI‐powered descriptions (via Ollama & Sentence‑Transformers), insert accessible tooltips into LaTeX source, compile to PDF, produce TTS audio, and export results as JSON & Word.

## Table of Contents

## Table of Contents

1. [Prerequisites](#prerequisites)  
2. [Installation](#installation)  
3. [Quick Start](#quick-start)    
4. [Module Reference](#module-reference)  
   - [1. Configuration & Initialization](#1-configuration--initialization)  
   - [2. File & Hash Utilities](#2-file--hash-utilities)  
   - [3. Image & PDF Conversion](#3-image--pdf-conversion)  
   - [4. LaTeX Text Extraction](#4-latex-text-extraction)  
   - [5. Text‐to‐Speech (TTS)](#5-text-to-speech-tts)  
   - [6. Chunking & Embeddings](#6-chunking--embeddings)  
   - [7. LLM Interaction (Ollama)](#7-llm-interaction-ollama)  
   - [8. LaTeX Utilities](#8-latex-utilities)  
   - [9. Code‐Figure Extraction](#9-code-figure-extraction)  
   - [10. Archiving & Compilation](#10-archiving--compilation)  
   - [11. Export (JSON & DOCX)](#11-export-json--docx)  
   - [12. High‐Level Pipelines](#12-high-level-pipelines)  

## Prerequisites  

- Python 3.8+  
- A local Ollama LLM server running at `http://localhost:11434`  
- Poppler (for `pdf2image`)  
- LaTeX toolchain (`pdflatex`)  
- Required Python packages (see [Installation](#installation))

## Installation

Libraries

In [ ]:

#--------------------------------------------------------------------------------------------------------------
# LIBRERIAS
#--------------------------------------------------------------------------------------------------------------

import os, re, sys, json, zipfile, subprocess, base64, logging, hashlib, shutil, tempfile
from pathlib import Path
from typing import List, Tuple, Dict, Any, Set, Union
import torch
import nltk
import numpy as np
from PIL import Image
from pydub import AudioSegment
import pyttsx3
from gtts import gTTS, gTTSError
from sentence_transformers import SentenceTransformer, util
from pdf2image import convert_from_path
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from pylatexenc.latex2text import LatexNodes2Text
from nltk.tokenize import sent_tokenize
from llm_service import LLMService
import config


## Quick Start

In [ ]:
from yourmodule import process_zip_and_generate_files

process_zip_and_generate_files(
    zip_path="path/to/your_archive.zip",
    json_filename="output/descriptions.json",
    word_filename="output/descriptions.docx"
)

This will:

Extract your_archive.zip.

Detect images & .tex files.

Decode embedded Base64 images.

Convert PDFs & images to PNG (deduplicated).

Extract plain text from LaTeX & build embeddings.

Generate three‐phase descriptions for pixel images & code figures.

Save JSON & Word reports.

Insert \pdftooltip{} tooltips into LaTeX, recompile PDF.

Produce MP3 via multi‐engine TTS.

Rezip the folder with all outputs.

## Module Reference

1. Configuration & Initialization

* NLTK Configuration
* Model initialization and LLM server verification

In [ ]:
# ─── Configuración NLTK ────────────────────────────────────────────────────────

nltk.download('punkt')

from nltk.tokenize import sent_tokenize

def fragment_text_to_sentences(text: str) -> List[str]:
    return sent_tokenize(text, language="english")



#--------------------------------------------------------------------------------------------------------------
# LLM_SERVICE Y CONFIGURACIONES
#--------------------------------------------------------------------------------------------------------------

llm = LLMService(model=config.LLM_MODEL)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
conversation_history: List[Dict[str, str]] = []


2. File & Hash Utilities

* create_unique_extraction_folder(zip_path: str) → str

Makes a new folder named extracted_< parts >.

In [ ]:
def create_unique_extraction_folder(zip_path: str) -> str:
    base_name = os.path.splitext(os.path.basename(zip_path))[0]
    words = re.split(r'\W+', base_name)
    folder_name = "extracted_" + "_".join(words[:4])
    folder_path = os.path.abspath(folder_name)
    os.makedirs(folder_path, exist_ok=True)
    return folder_path

* extract_files_from_zip(zip_path: str, extract_to: str) → None

Unpacks all entries.

In [ ]:
def extract_files_from_zip(zip_path: str, extract_to: str) -> None:
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_to)
    logging.info(f"Extraído '{zip_path}' → '{extract_to}'")

* find_images_and_tex_files(folder: str) → (List[str], List[str])

Finds image files (jpg, png, svg, pdf…) and .tex sources + includegraphics.

In [ ]:
def find_images_and_tex_files(folder: str) -> Tuple[List[str], List[str]]:
    image_exts = {'.jpg', '.jpeg', '.png', '.svg', '.pdf', '.eps', '.ps', '.tif', '.tiff'}
    image_paths = set()
    tex_files   = []

    # 1) Recorremos disco en busca de ficheros con extensión
    for root, _, files in os.walk(folder):
        for f in files:
            ext = os.path.splitext(f.lower())[1]
            path = os.path.abspath(os.path.join(root, f))
            if ext in image_exts:
                image_paths.add(path)
            elif ext == '.tex':
                tex_files.append(path)

    # 2) Buscamos includes sin extensión en cada .tex
    inc_pat = re.compile(r'\\includegraphics(?:\[[^\]]*\])?\{([^}]+)\}')
    for tex in tex_files:
        try:
            with open(tex, encoding='utf-8') as fh:
                text = fh.read()
        except Exception as e:
            logging.warning(f"No pude leer {tex}: {e}")
            continue

        for m in inc_pat.finditer(text):
            rel = m.group(1)
            base = os.path.dirname(tex)
            for ext in image_exts:
                cand = os.path.abspath(os.path.join(base, rel + ext))
                if os.path.exists(cand):
                    image_paths.add(cand)

    return list(image_paths), tex_files

* extract_base64_images_from_tex(tex_paths: List[str], output_folder: str) → List[str]

Decodes embedded Base64 images, deduplicates by MD5.

In [ ]:
def extract_base64_images_from_tex(tex_paths: List[str], output_folder: str) -> List[str]:
    os.makedirs(output_folder, exist_ok=True)
    saved_paths = []
    seen_hashes = set()
    # DOTALL para que [\s\S]+? incluya saltos de línea
    pattern = re.compile(r'data:image/(png|jpeg);base64,([\s\S]+?)', re.DOTALL)

    for tex in tex_paths:
        with open(tex, encoding='utf-8') as fh:
            content = fh.read()

        for i, m in enumerate(pattern.finditer(content)):
            fmt, data_b64 = m.group(1), m.group(2)
            # si es muy pequeño, es probable que no sea realmente una imagen
            if len(data_b64) < 1024:
                continue

            raw = base64.b64decode(data_b64)
            h = hashlib.md5(raw).hexdigest()
            if h in seen_hashes:
                continue
            seen_hashes.add(h)

            ext = 'jpg' if fmt == 'jpeg' else 'png'
            fn = f"embedded_{os.path.splitext(os.path.basename(tex))[0]}_{i}.{ext}"
            out_path = os.path.join(output_folder, fn)

            with open(out_path, 'wb') as imgf:
                imgf.write(raw)
            logging.info(f"Decodificada imagen base64 → {out_path}")
            saved_paths.append(out_path)

    return saved_paths

* compute_file_hash(file_path: str) → str

Returns MD5 hash (for deduplication).

In [ ]:
def compute_file_hash(file_path: str) -> str:
    md5 = hashlib.md5()
    try:
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(4096), b''):
                md5.update(chunk)
    except Exception as e:
        logging.error(f"Error de hash en '{file_path}': {e}")
        return ""
    return md5.hexdigest()

3. Image & PDF Conversion

* convert_image_to_png(image_path: str) → str

Loads via PIL, converts CMYK→RGB, saves PNG.

In [ ]:
def convert_image_to_png(image_path: str) -> str:
    try:
        img = Image.open(image_path)
        if img.mode == 'CMYK':
            img = img.convert('RGB')
        png_path = os.path.splitext(image_path)[0] + ".png"
        img.save(png_path, "PNG")
        logging.info(f"Convertida imagen → '{png_path}'")
        return png_path
    except Exception as e:
        logging.warning(f"No se pudo convertir {image_path}: {e}")
        return ""

* convert_pdf_to_png(pdf_path: str) → str

Renders first page at 200 dpi to PNG via pdf2image.

In [ ]:
def convert_pdf_to_png(pdf_path: str) -> str:
    try:
        from pdf2image import convert_from_path
        pages = convert_from_path(pdf_path, dpi=200)
        if pages:
            png_path = os.path.splitext(pdf_path)[0] + ".png"
            pages[0].save(png_path, "PNG")
            logging.info(f"Convertido PDF → '{png_path}'")
            return png_path
    except Exception as e:
        logging.error(f"Error conversión PDF '{pdf_path}': {e}")
    return ""

* reduce_image_size(image_path: str, max_size_bytes: int) → str

Iteratively lowers JPEG quality until under size limit.

In [ ]:
def reduce_image_size(image_path: str, max_size_bytes: int = 2 * 1024 * 1024) -> str:

    img = Image.open(image_path)

    # Convertir la imagen a RGB si tiene un canal alfa (RGBA)
    if img.mode == 'RGBA':
        img = img.convert('RGB')

    quality = 85  # Comienza con una calidad de 85

    # Crea una ruta temporal para guardar la imagen redimensionada
    temp_path = os.path.join(tempfile.gettempdir(), os.path.basename(image_path).replace(" ", "_"))

    # Reduce la calidad hasta que el tamaño del archivo sea adecuado
    while True:
        img.save(temp_path, format='JPEG', quality=quality)
        if os.path.getsize(temp_path) <= max_size_bytes or quality <= 10:
            break
        quality -= 5  # Disminuir calidad gradualmente hasta alcanzar el límite

    return temp_path

4. LaTeX Text Extraction

* extract_text_from_latex(main_tex: str) → (Dict[str,str], List[Dict], str)

• Cleans comments & \cite{}

• Extracts abstract

• Renumbers sections & resolves \ref{}

• Returns

sections: title→plain text

figures: list of {caption, pdftooltip}

abstract: raw abstract text

In [ ]:
def extract_text_from_latex(
    main_tex: str
) -> Tuple[Dict[str, str], List[Dict[str, str]], str]:

    raw = open(main_tex, 'r', encoding='utf-8').read()

    # 1) Limpia comentarios y citas
    clean = re.sub(r'(?<!\\)%.*', '', raw)
    clean = re.sub(r'\\cite\{[^}]*\}', '', clean)

    # 2) Extrae abstract
    mabs = re.search(r'\\begin\{abstract\}(.*?)\\end\{abstract\}', clean, re.DOTALL)
    abstract = mabs.group(1).strip() if mabs else ""

    # 3) Numeración de secciones (y recolección de labels)
    labels = {}
    def sec_repl(m):
        idx = len(labels) + 1
        lbl = m.group(2)
        if lbl: labels[lbl] = idx
        return m.group(1)
    clean = re.sub(r'(\\section\*?\{[^}]*\})(?:\\label\{([^}]*)\})?', sec_repl, clean)

    # 4) Reemplaza referencias por números
    clean = re.sub(r'\\ref\{([^}]*)\}', lambda m: str(labels.get(m.group(1), -1)), clean)
    clean = re.sub(r'\\eqref\{([^}]*)\}', lambda m: f"({labels.get(m.group(1),-1)})", clean)

    # 5) Divide en secciones
    parts = re.split(r'(\\section\*?\{([^}]*)\})', clean)
    sections = {}
    for i in range(1, len(parts), 3):
        title = parts[i+1]
        body = parts[i+2]
        text = LatexNodes2Text().latex_to_text(body)
        text = "\n".join(line.lstrip() for line in text.splitlines() if line.strip())
        sections[title] = text

    # 6) Figuras con pdftooltip
    figs = []
    for blk in re.findall(r'\\begin\{figure\}(.*?)\\end\{figure\}', clean, re.DOTALL):
        cap = re.search(r'\\caption\{([^}]*)\}', blk)
        tip = re.search(r'\\pdftooltip\{([^}]*)\}\{([^}]*)\}', blk)
        figs.append({
            'caption': cap.group(1) if cap else "",
            'pdftooltip': tip.group(2) if tip else ""
        })

    return sections, figs, abstract


* extract_text_from_tex(tex_files: List[str]) → str

Strips LaTeX commands (except \title, \section) to join all sources into plain text.

In [ ]:
def extract_text_from_tex(tex_files: List[str]) -> str:
    plain_parts = []
    for tex in tex_files:
        try:
            content = open(tex, 'r', encoding='utf-8').read()
            content = re.sub(r'\\(?!title|section)[A-Za-z]+\{.*?\}', '', content)
            plain_parts.append(content)
        except Exception as e:
            logging.warning(f"Error leyendo {tex}: {e}")
    logging.info("Extraído texto plano de .tex")
    return "\n".join(plain_parts)

5. Text‑to‑Speech (TTS)

* tts(content: str, idioma: str='en') → str
Multi‐fallback pipeline:

macOS say

Linux espeak

Coqui TTS
Returns MP3 path or "".



In [ ]:
def tts(contenido: str, idioma: str = 'en') -> str:
    import os, platform, tempfile, subprocess, logging, re
    from pydub import AudioSegment

    os.environ["TOKENIZERS_PARALLELISM"] = "false"

    out_mp3 = os.path.abspath("speechPDF.mp3")
    temp_dir = tempfile.gettempdir()
    max_len = 5000

    # Fragmentar texto largo
    sentences = re.split(r'(?<=[\.\?\!])\s+', contenido.strip())
    chunks, curr = [], ""
    for s in sentences:
        if len(curr) + len(s) + 1 > max_len:
            chunks.append(curr)
            curr = s
        else:
            curr = (curr + " " + s).strip()
    if curr:
        chunks.append(curr)

    # ─── Opción 1: macOS nativo 'say' con -f ───────────────────────────────
    if platform.system() == "Darwin":
        try:
            temp_paths = []
            for i, chunk in enumerate(chunks):
                textfile = os.path.join(temp_dir, f"say_text_{i}.txt")
                with open(textfile, "w") as f:
                    f.write(chunk)

                aiff_file = os.path.join(temp_dir, f"say_chunk_{i}.aiff")
                subprocess.run(['say', '-v', 'Samantha', '-f', textfile, '-o', aiff_file], check=True)
                temp_paths.append(aiff_file)
                os.remove(textfile)

            combined = AudioSegment.from_file(temp_paths[0], format="aiff")
            for path in temp_paths[1:]:
                combined += AudioSegment.from_file(path, format="aiff")
            combined.export(out_mp3, format="mp3")

            for path in temp_paths:
                os.remove(path)

            return out_mp3

        except Exception as e:
            logging.warning(f"tts(): 'say' en macOS falló ({e}), probando fallback...")

    # ─── Opción 2: espeak (Linux) ────────────────────────────────────────────
    if platform.system() == "Linux":
        try:
            temp_wavs = []
            for i, chunk in enumerate(chunks):
                temp_wav = os.path.join(temp_dir, f"espeak_chunk_{i}.wav")
                with open(temp_wav, 'wb') as wf:
                    subprocess.run(['espeak', '-v', idioma, '--stdout', chunk],
                                   stdout=wf, stderr=subprocess.DEVNULL, check=True)
                temp_wavs.append(temp_wav)

            combined = AudioSegment.from_wav(temp_wavs[0])
            for path in temp_wavs[1:]:
                combined += AudioSegment.from_wav(path)
            combined.export(out_mp3, format="mp3")

            for path in temp_wavs:
                os.remove(path)

            return out_mp3

        except Exception as e2:
            logging.warning(f"tts(): espeak falló ({e2}), probando Coqui TTS...")

    # ─── Opción 3: Coqui TTS ─────────────────────────────────────────────────
    try:
        from TTS.api import TTS as CoquiTTS
        coqui = CoquiTTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False, gpu=False)

        temp_wavs = []
        for i, chunk in enumerate(chunks):
            temp_wav = os.path.join(temp_dir, f"coqui_chunk_{i}.wav")
            coqui.tts_to_file(text=chunk, file_path=temp_wav)
            temp_wavs.append(temp_wav)

        combined = AudioSegment.from_wav(temp_wavs[0])
        for path in temp_wavs[1:]:
            combined += AudioSegment.from_wav(path)
        combined.export(out_mp3, format="mp3")

        for path in temp_wavs:
            os.remove(path)

        return out_mp3

    except Exception as e3:
        logging.error(f"tts(): Fallback con Coqui TTS falló ({e3})")

    return ""

6. Chunking & Embeddings

* chunk_text(text: str, max_words: int=200) → List[str]

Word‐based sliding windows for RAG.

In [ ]:
def chunk_text(text: str, max_words: int = 200) -> List[str]:
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunks.append(" ".join(words[i:i + max_words]))
    return chunks

* encode_no_grad(texts, **kwargs) → torch.Tensor

Embeds via Sentence‑Transformer under torch.no_grad(), clears GPU cache.

In [ ]:
def encode_no_grad(
    texts: Union[str, List[str]],
    **kwargs
) -> torch.Tensor:

    with torch.no_grad():
        embs = embedding_model.encode(texts, **kwargs)
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    return embs

7. LLM Interaction (Ollama)

* generate_text_adv(prompt: str, model: str="llama4:maverick") → str

Simple textual chat.

In [ ]:
def generate_text_adv(prompt: str, model: str = "llama4:maverick") -> str:

    try:
        resp = ollama.chat(
            model=model,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )
        # resp.message.content es la cadena de respuesta
        return resp.message.content.strip()
    except Exception as e:
        logging.error(f"[ollama.chat] Error en generate_text_adv: {e}")
        return "Fallback: " + prompt

* generate_text_rag_custom(prompt: str, contexts: List[str], ctx_embs: Tensor, q_emb: Tensor, top_k: int=3) → str

Cosine‐similarity to pick top_k contexts, appends them to prompt.

In [ ]:
def generate_text_rag_custom(
    prompt: str,
    contexts: List[str],
    ctx_embs: torch.Tensor,
    q_emb: torch.Tensor,
    top_k: int = 3
) -> str:
    if not contexts or ctx_embs.size(0) == 0:
        return generate_text_adv(prompt)
    sims = util.cos_sim(q_emb, ctx_embs)[0]
    k = min(top_k, sims.size(0))
    _, idx = torch.topk(sims, k)
    sel = [contexts[i] for i in idx.tolist()]
    enhanced = prompt + "\n\nRelevant contexts:\n" + "\n\n---\n\n".join(sel)
    return generate_text_adv(enhanced)

* llama_vision_request(prompt: str, image_path: str="", code: str="", model: str="llama4:maverick") → str

Multimodal: sends both text and optional image or code.

In [ ]:
def llama_vision_request(prompt: str, image_path: str = "", code: str = "", model: str = "llama4:maverick") -> str:

    # Preparamos la lista de imágenes si existe
    images = [image_path] if image_path else []

    # Si hay código, lo concatenamos al prompt
    if code:
        prompt = f"{prompt}\n\nCode:\n{code}"

    try:
        resp = ollama.chat(
            model=model,
            messages=[{
                "role": "user",
                "content": prompt,
                "images": images
            }]
        )
        return resp.message.content.strip()
    except Exception as e:
        logging.error(f"[ollama.chat] Error en llama_vision_request: {e}")
        return ""

8. LaTeX Utilities

* find_caption_for_image(image_name: str, tex_files: List[str]) → str

Regex‐searches the figure environment for a matching \caption{…}.



In [ ]:
  def find_caption_for_image(image_name: str, tex_files: List[str]) -> str:
    esc = re.escape(image_name)
    pattern = re.compile(
        rf"\\begin\{{figure\}}.*?{esc}.*?\\caption\{{([^}}]*)\}}.*?\\end\{{figure\}}",
        re.DOTALL
    )
    for tex in tex_files:
        try:
            content = open(tex, 'r', encoding='utf-8').read()
            m = pattern.search(content)
            if m:
                return m.group(1)
        except Exception:
            continue
    return ""

* ensure_caption_in_preamble(main_tex: str) → None

Inserts \usepackage{hyperref,pdfcomment,accsupp} before \begin{document} if missing.

In [ ]:
def ensure_caption_in_preamble(main_tex: str) -> None:

    p = Path(main_tex)
    content = p.read_text(encoding='utf-8')
    additions = []
    if r'\usepackage{hyperref}' not in content:
        additions.append(r'\usepackage{hyperref}')
    if r'\usepackage{pdfcomment}' not in content:
        additions.append(r'\usepackage{pdfcomment}')
    if r'\usepackage{accsupp}' not in content:
        additions.append(r'\usepackage{accsupp}')
    if not additions:
        return

    insert_block = "\n".join(additions) + "\n"
    # Buscamos dónde arranca el document
    m = re.search(r'\\begin\{document\}', content)
    if m:
        # Insertamos justo antes de \begin{document}
        new_content = content[:m.start()] + insert_block + content[m.start():]
    else:
        # Si no lo encuentra, lo ponemos al inicio
        new_content = insert_block + content

    p.write_text(new_content, encoding='utf-8')


* insert_descriptions_into_tex(tex_files: List[str], final_output: Dict) → None

Wraps each \caption{…} in \pdftooltip{visible}{description} using the JSON descriptions.

In [ ]:
def insert_descriptions_into_tex(
    tex_files: list[str],
    final_output: dict[str, list[dict]]
) -> None:

    # 1) Construir mapa caption→descripcion
    desc_map = {}
    for cat in ("pixel_images", "code_figures"):
        for item in final_output.get(cat, []):
            cap = item.get("caption", "").strip()
            d3  = item.get("description_3", "").strip()
            if cap and d3:
                key = re.sub(r'\s+', ' ', cap.lower())
                desc_map[key] = d3

    caption_re = re.compile(
        r'(\\caption(?:\[[^\]]*\])?\{)'  # grupo1: \caption...[{
        r'((?:[^{}]|{[^}]*})+)'          # grupo2: contenido
        r'\}',                           # cierre
        re.DOTALL
    )

    for fp in tex_files:
        p    = Path(fp)
        orig = p.read_text(encoding='utf-8')

        def repl(m: re.Match) -> str:
            visible = m.group(2)
            key     = re.sub(r'\s+', ' ', visible.lower())
            desc    = desc_map.get(key, "")
            if not desc:
                return m.group(0)
            esc = format_tooltip_text(desc)
            # cada '{' literal es '{{' y cada '}' literal es '}}'
            return (
                f"{m.group(1)}"
                f"\\pdftooltip{{{visible}}}{{{esc}}}}}"
            )

        new = caption_re.sub(repl, orig)
        if new != orig:
            bak = p.with_suffix(p.suffix + '.bak')
            if not bak.exists():
                shutil.copyfile(p, bak)
            p.write_text(new, encoding='utf-8')
            logging.info(f"Inserted tooltips into '{fp}'")


* escape_latex_special_chars(text: str) → str

Escapes \ & % $ # _ { } ~ ^ for safe LaTeX embedding.

In [ ]:
def escape_latex_special_chars(text: str) -> str:

    replacements = {
        '\\': r'\textbackslash{}',
        '&':  r'\&',
        '%':  r'\%',
        '$':  r'\$',
        '#':  r'\#',
        '_':  r'\_',
        '{':  r'\{',
        '}':  r'\}',
        '~':  r'\textasciitilde{}',
        '^':  r'\^{}',
    }
    for char, esc in replacements.items():
        text = text.replace(char, esc)
    return text

* format_tooltip_text(md_text: str) → str

Converts markdown bold → \textbf{}, collapses line breaks, then escapes.

In [ ]:
def format_tooltip_text(md_text: str) -> str:

    # 1) Negritas → \textbf
    txt = re.sub(r'\*\*(.*?)\*\*', r'\\textbf{\1}', md_text)
    # 2) Normalizar saltos
    txt = txt.replace('\r\n', '\n').replace('\r', '\n')
    txt = txt.replace('\n\n', r'\\newline\\newline ')
    txt = txt.replace('\n', ' ')
    # 3) Escape total
    return escape_latex_special_chars(txt.strip())

* normalize_image_reference(name: str) → str
  
Strips non‑alphanumeric characters and lowercases an image filename (without extension) to a “normalized” key for matching.  



In [ ]:
def normalize_image_reference(name: str) -> str:
    base = os.path.splitext(os.path.basename(name))[0].lower()
    return re.sub(r'\W+', '', base)

* get_relevant_paragraphs_for_image(ref_id: str, plain_text: str) → str  

Given an image reference ID and the document’s plain text, returns any paragraphs that contain the normalized key or common figure‑related keywords (`see`, `shown`, `as seen`).  


In [ ]:
def get_relevant_paragraphs_for_image(ref_id: str, plain_text: str) -> str:
    norm = normalize_image_reference(ref_id)
    paras = plain_text.split("\n\n")
    relevant = []
    for p in paras:
        low = re.sub(r'\W+', '', p.lower())
        if norm in low or ("figure" in p.lower() and any(k in p.lower() for k in ["see", "shown", "as seen"])):
            relevant.append(p)
    return "\n\n".join(relevant)

* normalize(s: str) → str

Removes all non‑alphanumeric characters and lowercases a string—used internally by multiple utilities for key matching.  



In [ ]:
def normalize(s: str) -> str:

    return re.sub(r'\W+', '', s.strip().lower())

9. Code‑Figure Extraction & Cleaning

* extract_figures_and_text(tex_path: str) → (List[Dict], str)

Parses figure blocks for TikZ/SVG/embedded images, returns list of {caption,label,format,filename,code} and the plain text without figures.

In [ ]:
def extract_figures_and_text(tex_path: str) -> Tuple[List[Dict[str, Any]], str]:

    from pylatexenc.latex2text import LatexNodes2Text

    # 1) Leer y limpiar comentarios e hipervínculos
    raw = Path(tex_path).read_text(encoding='utf-8')
    lines = []
    for line in raw.splitlines():
        if not line.strip().startswith('%'):
            lines.append(line)
    cleaned = "\n".join(lines)
    cleaned = re.sub(r'\\href\{[^\}]*\}\{[^\}]*\}', '', cleaned, flags=re.DOTALL)

    # 2) Extraer bloques figure[*] con opcionales [<opts>]
    fig_block_re = re.compile(
        r'\\begin\{figure\*?\}(?:\[[^\]]*\])?'  # begin{figure} o figure* y opcionales [..]
        r'(.*?)'
        r'\\end\{figure\*?\}',
        re.DOTALL
    )

    figures: List[Dict[str, Any]] = []
    count = 0
    dirpath = Path(tex_path).parent

    for blk in fig_block_re.findall(cleaned):
        count += 1
        entry: Dict[str, Any] = {'Figure': count}

        # caption
        cap_m = re.search(r'\\caption(?:\[[^\]]*\])?\{([^}]*)\}', blk, re.DOTALL)
        entry['caption'] = cap_m.group(1).strip() if cap_m else ""

        # label
        lbl_m = re.search(r'\\label\{([^}]*)\}', blk)
        if lbl_m:
            entry['label'] = lbl_m.group(1).strip()

        # incluye SVG
        svg_m = re.search(r'\\includesvg(?:\[[^\]]*\])?\{([^}]*)\}', blk)
        # detecta TikZ
        tikz_m = re.search(r'\\begin\{tikzpicture\}', blk)
        # incluye gráficos convencionales
        img_m  = re.search(r'\\includegraphics(?:\[[^\]]*\])?\{([^}]*)\}', blk)

        if svg_m:
            fn = svg_m.group(1).strip()
            entry.update({'format': 'SVG', 'filename': fn})
            # intenta leer código SVG/TEX/TIKZ asociado
            code = ""
            for ext in ('.svg', '.tex', '.tikz'):
                cand = dirpath / (fn + ext)
                if cand.is_file():
                    code = cand.read_text(encoding='utf-8')
                    break
            entry['code'] = code or blk.strip()

        elif tikz_m:
            entry.update({'format': 'TikZ', 'filename': ''})
            entry['code'] = blk.strip()

        elif img_m:
            fn = img_m.group(1).strip()
            entry.update({'format': 'Embedded', 'filename': fn})
            entry['code'] = blk.strip()

        else:
            entry.update({'format': 'Unknown', 'filename': ''})
            entry['code'] = blk.strip()

        figures.append(entry)

    # 3) Construir plain_text sin los bloques figure
    without_figs = fig_block_re.sub('', cleaned)
    plain_text = LatexNodes2Text().latex_to_text(without_figs)
    plain_text = "\n".join(line.strip() for line in plain_text.splitlines() if line.strip())

    return figures, plain_text


* remove_commented_lines(content: str) → str

Strips lines starting with %.

In [ ]:
def remove_commented_lines(content: str) -> str:
    return "\n".join([l for l in content.splitlines() if not l.strip().startswith('%')])


* read_input_file(base_path: str) → str

Loads .tikz, .tex or .svg code from disk.

In [ ]:
def read_input_file(base_path: str) -> str:
    for ext in ['.tikz', '.tex', '.svg']:
        candidate = base_path + ext
        if os.path.exists(candidate):
            try:
                return open(candidate, 'r', encoding='utf-8').read()
            except:
                return ""
    return ""

* clean_description(desc: str) → str  

Removes all LaTeX/math syntax (`$…$`), converts primes (e.g. `Q'` → “Q prime”) and subscripts (`x_1` → “x sub 1”), optionally renames standalone `Q` to “quiver”, and collapses extra spaces—yielding plain English text.  


In [ ]:
def clean_description(desc: str) -> str:

    # 1) Quita delimitadores matemáticos $...$
    desc = re.sub(r'\$(.*?)\$', r'\1', desc)
    # 2) Convierte primados: e.g. Q' → Q prime
    desc = re.sub(r"\b([A-Za-z]+)'", r"\1 prime", desc)
    # 3) Convierte subscritos: x_1 → x sub one, x_12 → x sub twelve
    def sub_to_words(m):
        num = m.group(1)
        # opcional: mapear 1->one, 2->two, etc. o dejar como dígito
        words = num  # o usar un dict para “one”, “two”…
        return f"x sub {words}"
    desc = re.sub(r'\bx_([0-9]+)\b', sub_to_words, desc)
    # 4) Opcionalmente renombrar 'Q' standalone a 'quiver'
    desc = re.sub(r'\bQ\b', 'quiver', desc)
    # 5) Limpiar espacios extra
    return re.sub(r'\s{2,}', ' ', desc).strip()

10. Archiving & Compilation

* rezip_folder(folder: str, out_zip: str, extra_files: List[str]=[]) → str

Zips entire folder plus any extras (PDF, etc.).

In [ ]:
def rezip_folder(folder: str, out_zip: str, extra_files: List[str] = []) -> str:

    zip_path = os.path.abspath(out_zip)
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(folder):
            for f in files:
                full = os.path.join(root, f)
                arc = os.path.relpath(full, folder)
                zf.write(full, arc)
        # Agregar archivos externos (ej. PDF)
        for ext_file in extra_files:
            if os.path.exists(ext_file):
                zf.write(ext_file, os.path.basename(ext_file))
    logging.info(f"ZIP generado con PDF: '{zip_path}'")
    return zip_path


* compile_latex_to_pdf(main_tex: str, work_dir: str) → str

Runs pdflatex twice, returns the resulting PDF path.



In [ ]:
def compile_latex_to_pdf(main_tex: str, work_dir: str) -> str:

    cwd = os.path.dirname(main_tex) or work_dir
    name = os.path.basename(main_tex)
    try:
        for _ in range(2):
            subprocess.run(
                ['pdflatex', '-interaction=nonstopmode', name],
                cwd=cwd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
            )
        pdf = os.path.join(cwd, os.path.splitext(name)[0] + '.pdf')
        return pdf if os.path.isfile(pdf) else ""
    except Exception as e:
        logging.error(f"Error compilando {main_tex}: {e}")
        return ""

11. Export (JSON & DOCX)

* save_details_to_json(details: Dict, json_file: str) → None

Flattens pixel & code figure descriptions to a single JSON list.

In [ ]:
def save_details_to_json(details: Dict[str,Any], json_file: str) -> None:
    seen = set()
    out = []
    for cat in ("pixel_images","code_figures"):
        for f in details.get(cat, []):
            name = f.get("image_name","")
            if not name or name in seen:
                continue
            seen.add(name)
            if cat == "pixel_images":
                rec = {
                    "caption":       f.get("caption",""),
                    "image_name":    name,
                    "image_path":    f.get("image_path",""),
                    "description_1": f.get("description_1",""),
                    "description_2": f.get("description_2",""),
                    "description_3": f.get("description_3","")
                }
            else:
                rec = {
                    "caption":       f.get("caption",""),
                    "image_name":    name,
                    "image_path":    f.get("image_path",""),
                    "description_1": f.get("description_1",""),
                    "description_2": f.get("description_2",""),
                    "description_3": f.get("description_3","")
                }
            out.append(rec)
    with open(json_file, 'w', encoding='utf-8') as jf:
        json.dump(out, jf, indent=4, ensure_ascii=False)
    logging.info(f"JSON guardado → {json_file}")

* save_details_to_word(details: Dict, word_file: str, max_w: float=4.5) → None

Builds a python-docx report: one table per figure with images & three descriptions.

In [ ]:
def save_details_to_word(details: Dict[str, Any], word_file: str, max_w: float = 4.5) -> None:
    from docx import Document
    from docx.shared import Inches, Pt, RGBColor
    from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
    from docx.enum.table import WD_TABLE_ALIGNMENT
    import os
    import logging

    doc = Document()
    seen = set()

    all_items = details.get("pixel_images", []) + details.get("code_figures", [])

    for item in all_items:
        name = item.get("image_name", "")
        if not name or name in seen:
            continue
        seen.add(name)

        caption = item.get("caption", name)
        image_path = item.get("image_path", "")
        desc1 = item.get("description_1", "")
        desc2 = item.get("description_2", "")
        desc3 = item.get("description_3", "")

        # Crear tabla con bordes visibles
        table = doc.add_table(rows=5, cols=2)
        table.alignment = WD_TABLE_ALIGNMENT.CENTER
        table.autofit = True
        set_table_borders(table)

        # Fila 1: Título de la figura
        table.cell(0, 0).merge(table.cell(0, 1))
        cell = table.cell(0, 0)
        p = cell.paragraphs[0]
        run = p.add_run(f"Figure: {caption}")
        run.font.name = 'Times New Roman'
        run.font.size = Pt(11)

        # Fila 2: Imagen centrada
        table.cell(1, 0).merge(table.cell(1, 1))
        p = table.cell(1, 0).paragraphs[0]
        run = p.add_run()
        if image_path and os.path.isfile(image_path):
            try:
                run.add_picture(image_path, width=Inches(max_w))
                p.alignment = WD_PARAGRAPH_ALIGNMENT.CENTER
            except Exception as e:
                logging.warning(f"Error insertando imagen {image_path}: {e}")
                p.add_run("(Error al insertar imagen)").font.size = Pt(11)

        # Fila 3: Descripción 1
        p = table.cell(2, 0).paragraphs[0]
        run = p.add_run("Description 1:")
        run.font.name = 'Times New Roman'
        run.font.size = Pt(11)
        run.font.color.rgb = RGBColor(0, 0, 255)  # azul

        p = table.cell(2, 1).paragraphs[0]
        run = p.add_run(desc1)
        run.font.name = 'Times New Roman'
        run.font.size = Pt(11)

        # Fila 4: Descripción 2 (RAG)
        p = table.cell(3, 0).paragraphs[0]
        run = p.add_run("Description 2 (RAG):")
        run.font.name = 'Times New Roman'
        run.font.size = Pt(11)
        run.font.color.rgb = RGBColor(0, 0, 255)  # azul

        p = table.cell(3, 1).paragraphs[0]
        run = p.add_run(desc2)
        run.font.name = 'Times New Roman'
        run.font.size = Pt(11)

        # Fila 5: Descripción 3 (Detailed)
        p = table.cell(4, 0).paragraphs[0]
        run = p.add_run("Description 3 (Detailed):")
        run.font.name = 'Times New Roman'
        run.font.size = Pt(11)
        run.font.color.rgb = RGBColor(0, 0, 255)  # azul

        p = table.cell(4, 1).paragraphs[0]
        run = p.add_run(desc3)
        run.font.name = 'Times New Roman'
        run.font.size = Pt(11)

        doc.add_paragraph()  # espacio
        doc.add_page_break()

    try:
        doc.save(word_file)
        logging.info(f"Word guardado → {word_file}")
    except Exception as e:
        logging.error(f"Error guardando Word: {e}")

12. High‑Level Pipelines

* process_pixel_images(images: List[str], tex_files: List[str], plain_text: str, contexts: List[str], context_embs: Tensor) → List[Dict]

Three‐phase AI descriptions for pixel images: exact, RAG‐improved, final synthesis.

In [ ]:
def process_pixel_images(
    images: List[str],
    tex_files: List[str],
    plain_text: str,
    contexts: List[str],
    context_embs: torch.Tensor
) -> List[Dict[str, Any]]:

    details: List[Dict[str, Any]] = []
    seen_hashes: Set[str] = set()

    for img in images:
        # Evitar duplicados por hash
        h = compute_file_hash(img)
        if not h or h in seen_hashes:
            continue
        seen_hashes.add(h)

        # Redimensionar la imagen si es necesario
        img = reduce_image_size(img)

        # 1) Descripción exacta
        desc1 = llama_vision_request(
            "Generate a precise, technical description of the visible objects, structures, and textures in the image. "
            "Focus on key elements such as shapes, colors, and spatial relationships, in third person. "
            "Provide the description in a single, continuous paragraph with no line breaks, bullet points, or special characters.",
            image_path=img
        )

        # 2) Contexto RAG combinado
        ctx = get_relevant_paragraphs_for_image(os.path.basename(img), plain_text) or ""
        q_emb = encode_no_grad(ctx, convert_to_tensor=True)

        # Definir top_context antes de usarlo
        top_context = generate_text_rag_custom(
            prompt="Improve the following technical description by incorporating the provided context. "
            "Ensure that the enhanced description is coherent, relevant, and directly related to the image. "
            "Focus on adding missing details or clarifying ambiguous aspects, using third person. "
            "Provide a concise, continuous paragraph with no line breaks, lists, or special characters.\n\n"
            f"Initial description:\n{desc1}\n\n"
            f"Context:\n{ctx}",
            contexts=contexts,
            ctx_embs=context_embs,
            q_emb=q_emb,
            top_k=3
        )

        # 2) Mejora del contexto con RAG
        desc2 = llama_vision_request(
            "Improve the following technical description by incorporating the provided context. "
            "Ensure that the enhanced description is coherent, relevant, and directly related to the image. "
            "Focus on adding missing details or clarifying ambiguous aspects, using third person. "
            "Provide a concise, continuous paragraph with no line breaks, lists, or special characters.\n\n"
            f"Initial description:\n{desc1}\n\n"
            f"Context:\n{top_context}",
            image_path=img
        )

        # 3) Síntesis final
        desc3 = llama_vision_request(
            "Synthesize a final, concise, and comprehensive technical description of the image by combining the initial, improved, and contextual descriptions. "
            "Ensure that the final description integrates all relevant details while avoiding redundancy or overly general statements. "
            "Provide the description in a single, continuous paragraph with no line breaks, lists, or special characters.\n\n"
            f"Initial description:\n{desc1}\n\n"
            f"Improved description:\n{desc2}\n\n"
            f"Context:\n{top_context}",
            image_path=img
        )

        # Construcción del registro de salida
        details.append({
            "caption":       find_caption_for_image(os.path.basename(img), tex_files),
            "image_name":    os.path.basename(img),
            "image_path":    img,
            "description_1": desc1,
            "description_2": desc2,
            "description_3": desc3
        })

    return details

* process_code_figures(tex_files: List[str], contexts: List[str], context_embs: Tensor) → List[Dict]

Three‐phase AI descriptions for TikZ/SVG code figures, then cleaned.

In [ ]:
def process_code_figures(
    tex_files: List[str],
    contexts: List[str],
    context_embs: torch.Tensor
) -> List[Dict[str, Any]]:

    code_details: List[Dict[str, Any]] = []
    seen = set()

    for tex in tex_files:
        figs, _ = extract_figures_and_text(tex)
        for fig in figs:
            fmt = fig.get("format", "")
            if fmt not in ("SVG", "TikZ"):
                continue

            name = fig.get("filename") or f"Figure{fig.get('Figure')}"
            if name in seen:
                continue
            seen.add(name)

            code = fig.get("code", "")
            caption = fig.get("caption", "")

            # Fase 1
            raw1 = llama_vision_request(
                prompt=(
                    "Describe and Generate a technical description of the following figure code in plain English. "
                    "Avoid LaTeX and math symbols—use words like 'vertex one' instead of x₁. "
                    + code
                ),
                code=code
            )
            desc1 = clean_description(raw1)

            # Fase 2
            raw2 = llama_vision_request(
                prompt=(
                    "Improve the previous description using this context, still in plain English:\n\n"
                    f"Initial description:\n{desc1}\n\n"
                    "Context:\n" + "\n\n".join(contexts[:3])
                ),
                code=code
            )
            desc2 = clean_description(raw2)

            # Fase 3
            raw3 = llama_vision_request(
                prompt=(
                    "Describe and Generate a technical description of the following figure code in plain English, no symbols and use the previous descriptions to enrich the description\n\n"
                    f"1) {desc1}\n\n"
                    f"2) {desc2}"
                ),
                code=code
            )
            desc3 = clean_description(raw3)

            code_details.append({
                "caption":       caption,
                "image_name":    name,
                "image_path":    "",
                "description_1": desc1,
                "description_2": desc2,
                "description_3": desc3
            })

    return code_details

* process_zip_and_generate_files(zip_path: str, json_filename: str, word_filename: str) → None

Orchestrates the complete workflow from ZIP → JSON/DOCX/PDF/MP3 → new ZIP.

In [ ]:
def process_zip_and_generate_files(
    zip_path: str,
    json_filename: str,
    word_filename: str
) -> None:

    # 1) Extrae el ZIP
    extract_to = create_unique_extraction_folder(zip_path)
    extract_files_from_zip(zip_path, extract_to)

    # 2) Detecta imágenes y .tex
    image_files, tex_files = find_images_and_tex_files(extract_to)
    base64_imgs = extract_base64_images_from_tex(
        tex_files,
        os.path.join(extract_to, "embedded_imgs")
    )

    # 3) Convierte todo a PNG y deduplica
    converted = []
    for p in image_files + base64_imgs:
        ext = os.path.splitext(p)[1].lower()
        if ext == '.pdf':
            new_p = convert_pdf_to_png(p)
        elif ext != '.png':
            new_p = convert_image_to_png(p)
        else:
            new_p = p
        if new_p:
            converted.append(os.path.abspath(new_p))
    images = list(dict.fromkeys(converted))

    # 4) Extrae texto plano
    plain_text = extract_text_from_tex(tex_files)

    # — Chunking + embeddings sin gradientes —
    chunks     = chunk_text(plain_text, max_words=200)
    chunk_embs = encode_no_grad(chunks, convert_to_tensor=True)

    # — Sentence splitting y embeddings sin gradientes —
    sentences = fragment_text_to_sentences(plain_text)
    sentences = sorted(sentences, key=lambda s: len(s), reverse=True)[:200]
    sent_embs  = encode_no_grad(sentences, convert_to_tensor=True)

    # — Combina contextos y embeddings —
    all_contexts = chunks + sentences
    all_embs     = torch.cat([chunk_embs, sent_embs], dim=0)

    # 5) Genera descripciones usando el contexto combinado
    pixel_details = process_pixel_images(
        images, tex_files, plain_text, all_contexts, all_embs
    )
    code_details  = process_code_figures(
        tex_files, all_contexts, all_embs
    )

    final_output = {
        "pixel_images": pixel_details,
        "code_figures": code_details
    }

    # 6) Guarda JSON y Word
    save_details_to_json(final_output, json_filename)
    save_details_to_word(final_output, word_filename)

    # 7) Inserta tooltips en TEX, recompila y genera audio
    if tex_files:
        main_tex = tex_files[0]
        ensure_caption_in_preamble(main_tex)
        insert_descriptions_into_tex(tex_files, final_output)

        pdf_file = compile_latex_to_pdf(main_tex, extract_to)
        sections, figures, abstract = extract_text_from_latex(main_tex)

        # Construye texto completo para TTS
        full_text = ""
        if abstract:
            full_text += "Abstract:\n" + abstract.strip() + "\n\n"
        for title, body in sections.items():
            full_text += f"{title.strip()}:\n{body.strip()}\n\n"

        # Intentar generar audio sin interrumpir el flujo
        audio_path = ""
        if full_text.strip():
            try:
                audio_path = tts(full_text, idioma='es')
                if audio_path:
                    logging.info(f"Audio generado → '{audio_path}'")
                else:
                    logging.info("Se saltó la generación de audio por error en TTS.")
            except Exception as e:
                logging.warning(f"Error inesperado en TTS: {e}")

        # Empaqueta todo en un nuevo ZIP (siempre)
        new_zip = rezip_folder(
            extract_to,
            os.path.splitext(os.path.basename(zip_path))[0]
            + "_with_descriptions.zip",
            extra_files=[pdf_file] if pdf_file else []
        )
        logging.info(f"Proceso finalizado: ZIP → '{new_zip}', PDF → '{pdf_file}'")


* set_table_borders(table) → None

Given a `python-docx` table, adds single‑line black borders on all sides and inner lines by manipulating the table’s XML (`w:tblBorders`).  

In [ ]:
def set_table_borders(table):
    tbl = table._tbl
    tblPr = tbl.tblPr
    if tblPr is None:
        tblPr = OxmlElement('w:tblPr')

    borders = OxmlElement('w:tblBorders')
    for border_name in ['top', 'left', 'bottom', 'right', 'insideH', 'insideV']:
        border = OxmlElement(f'w:{border_name}')
        border.set(qn('w:val'), 'single')
        border.set(qn('w:sz'), '8')        # grosor
        border.set(qn('w:space'), '0')
        border.set(qn('w:color'), '000000')  # negro
        borders.append(border)

    tblPr.append(borders)

    if tbl.tblPr is None:
        tbl._element.append(tblPr)